# NMR vs EPR vs muSR

Visualizing differences in ZFS signatures for different techniques (NMR, EPR, muSR) in transverse field (TF) experiments using S=1/2, I=1/2 system

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xarray as xr
import netCDF4 as nc
from utils import stick_spectrum

import Python.plot_settings
pio.templates.default = "DemonLab"


In [ ]:
def add_doubleheaded_arrow(fig, x1, x2, y, label, label_offset=-2, font_size=20):
    # Add line (arrow body)
    fig.add_shape(
        type="line",
        x0=x1, y0=y,
        x1=x2, y1=y,
        line=dict(color="black", width=1.5),
        yref="paper",
    )

    # Add left arrowhead (pointing right)
    fig.add_annotation(
        x=x1, y=y,
        ax=x1 + 20, ay=y,
        showarrow=True,
        arrowhead=3,
        arrowsize=1.2,
        arrowwidth=1.5,
        arrowcolor="black",
        yref="paper",
    )

    # Add right arrowhead (pointing left)
    fig.add_annotation(
        x=x2, y=y,
        ax=x2 - 20, ay=y,
        showarrow=True,
        arrowhead=3,
        arrowsize=1.2,
        arrowwidth=1.5,
        arrowcolor="black",
        yref="paper",
    )

    # Add centered text with paper-relative offset
    fig.add_annotation(
        x=(x1 + x2) / 2,
        y=y,
        yshift=label_offset,
        text=label,
        showarrow=False,
        font=dict(size=font_size),
        yref="paper",
    )

def get_concoluted_spectrum(freq, amps, freq_axis, gamma):
    """
    freq: array of peak frequencies
    amps: array of peak amplitudes
    freq_axis: frequency axis for plotting
    gamma: FWHM of each Lorentzian peak
    """
    def lorentzian(x, x0, amp, gamma):
        return amp * (0.5*gamma)**2 / ((x - x0)**2 + (0.5*gamma)**2)

    spectrum = np.zeros_like(freq_axis)
    for f, a in zip(freq, amps):
        if np.isfinite(f) and np.isfinite(a):
            spectrum += lorentzian(freq_axis, f, a, gamma)
    return spectrum

# Problem of MathJax rendering, it can't go beyond font size 20
# therefore use r'$\Large ...$' for larger symbols
def create_base_figure():
    fig = go.Figure()
    fig.update_layout(
        xaxis_title='𝜈',    # or use r'$\Large \nu$'
        yaxis_title='Amplitude',
        xaxis=dict(
            showticklabels=False,   # remove x ticks
            showgrid=False,
            ticks='',               # remove tick marks
            title_standoff=8,        # decrease distance between x label and axis
            minor=dict(ticks='', showgrid=False)  # remove minor ticks and minor grid
        ),
        margin=dict(l=90, r=20, t=20, b=40),
        height=400, width=600,
    )
    return fig


## Zero Field Splitting


### NMR

#### Only dipolar coupling of D_parallel = 2 MHz
expect 2 peaks with a splitting of 3D

In [ ]:
xarr = xr.load_dataset('Data/TF_NMR_dipolar_cc.nc')
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2, 2
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[0], freq[1], 0.8, r'$\Large \frac{3}{2}D_\parallel$', label_offset=-2, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/NMR_dipolar_ZFS.pdf')


#### Only anisotropic hyperfine coupling of A_iso = 2 MHz

In [ ]:
xarr = xr.load_dataset('Data/TF_NMR_isotropic_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -3, 3
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))

fig.show()
# fig.write_image('../../Figures/Manuscript/TF_simulations/NMR_iso_ZFS.pdf')


### muSR

#### Only dipolar coupling of D_parallel = 2 MHz
expect 4 peaks with 1/2 D splitting, 2 peaks in middle are SQE transitions which are invisible in NMR

In [ ]:
xarr = xr.load_dataset('Data/TF_muSR_dipolar_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2, 2
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[0], freq[2], 0.8, r'$\Large \frac{1}{2}D$', label_offset=-2, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/muSR_dipolar_ZFS.pdf')


#### Only isotropic hyperfine coupling of A_iso = 2 MHz

3 peaks with A splitting

In [ ]:
xarr = xr.load_dataset('Data/TF_muSR_isotropic_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -3, 3
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[1], freq[0], 0.45, r'$\Large A_\text{iso}$', label_offset=-12, font_size=20)
add_doubleheaded_arrow(fig, freq[2], freq[3], 0.45, r'$\Large A_\text{iso}$', label_offset=-12, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/muSR_iso_ZFS.pdf')


### EPR zero field splitting


In [ ]:
xarr = xr.load_dataset('Data/TF_EPR_dipolar_cc.nc')
# print(ds)
theta = 0
B = 0

freq = xarr['frequencies'].sel(B=B, theta=theta).values * 1e3 # Convert to GHz
amps = xarr['amplitudes'].sel(B=B, theta=theta).values

# Define the frequency axis for plotting
freq_min, freq_max = -2, 2
n_points = 8000
freq_axis = np.linspace(freq_min, freq_max, n_points)
gamma = 0.01  # MHz

spectrum = get_concoluted_spectrum(freq, amps, freq_axis, gamma)

# Plot
fig = create_base_figure()
fig.add_trace(go.Scatter(x=freq_axis, y=spectrum, mode='lines', line=dict(color='black', width=2)))
add_doubleheaded_arrow(fig, freq[0], freq[2], 0.8, r'$\Large \frac{1}{2}D_\parallel$', label_offset=-2, font_size=20)

fig.show(renderer='browser')
# fig.write_image('../../Figures/Manuscript/TF_simulations/EPR_dipolar_ZFS.pdf')


# Avoided level-crossing (with only anisotropic dipolar coupling)

# Pake Pattern for powder sample